In [1]:
# uncomment the following line if you are running this notebook on Google Colab
# this will install the most recent version of TensorFlow and Keras 3
# !pip install tensorflow --upgrade

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import keras
print(keras.__version__)

In [4]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score

In [5]:
# uncomment the following line to prevent some possible errors
# !pip install --upgrade --no-cache-dir gdown

In [6]:
# uncomment the following line to download the dataset if you haven't already
!gdown 1-0d315aj7Ai8NNqat65XDvaOcHDcHiUD

In [7]:
# uncomment the following line to unzip the dataset if you didn't do it before
!unzip famous_paintings.zip > /dev/null 2>&1

In [8]:
dir_name = 'data/*'
files = glob.glob(dir_name)
df_train = pd.DataFrame({'full_path': files})
painters = [name.split('/')[1].split('.')[0] for name in df_train['full_path']] # TODO: get the painters' names from the file names
painters = [name.replace('_' + name.split('_')[-1], '') for name in painters]
df_train['painter'] = painters
df_train.head()

In [9]:
# get the list of unique painters in the dataset
class_names = df_train.painter.unique()
print(class_names)

In [10]:
fig, axes = plt.subplots(3, 4, figsize=(15, 15))
random_indices = np.random.choice(df_train.index, size=12, replace=False)
for i, ax in zip(random_indices, axes.flatten()):
    img = keras.preprocessing.image.load_img(df_train.full_path[i], target_size=(224, 224))
    ax.imshow(img)
    ax.set_title(df_train.painter[i])
    ax.axis('off')
plt.tight_layout()
plt.show()

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['painter'])# TODO: encode the painters' names
df_train.head()

In [12]:
print(df_train['label'].unique().min())
print(df_train['label'].unique().max())

In [13]:
import shutil

# create a folder for each label
for label in df_train.label.unique():
    os.makedirs(f'data/{label}', exist_ok=True)

# move each image to its corresponding label folder
for i, row in df_train.iterrows():
    shutil.move(row.full_path, f'data/{row.label}')

In [14]:
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
  directory= './data/',
  labels='inferred',
  label_mode='categorical',
  color_mode='rgb',
  batch_size=64,
  image_size=(256, 256),
  interpolation='bilinear',
  shuffle=True,
  seed=42,
  validation_split=0.2,
  subset='training'
)

# find the class names so in prediction time we can map the predictions to the painters properly
class_names = train_dataset.class_names
print('Class names:', class_names)

val_dataset = image_dataset_from_directory(
  directory= './data/',
  labels='inferred',
  label_mode='categorical',
  color_mode='rgb',
  batch_size=64,
  image_size=(256, 256),
  interpolation='bilinear',
  shuffle=False,
  validation_split=0.2,
  subset='validation'
)

In [15]:
from keras.applications.resnet50 import preprocess_input
from keras.utils import to_categorical

# Preprocess the data
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y)) # TODO: apply the preprocess_input function to the train_dataset
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), y)) # TODO: apply the preprocess_input function to the val_dataset

In [16]:
import plotly.express as px

# metric: 'accuracy' or 'loss'
def display_curves(history, metric):
  df = pd.DataFrame(history.history[metric], columns=[metric])
  df['val_'+metric] = history.history['val_'+metric]
  fig = px.line(df, x= df.index+1, y= [metric, 'val_'+metric])
  fig.update_layout(xaxis_title='Epochs', yaxis_title=metric)
  fig.show()

In [17]:
base_model = keras.applications.ResNet50(
    input_shape= (256, 256, 3),
    classes= 25,
    weights= 'imagenet',
    include_top= False,
    pooling= 'avg' # TODO
)

# Complete the model based on your choice of the architecture

base_model.trainable = False

for layer in base_model.layers[-5 : ]:
  layer.trainable = True

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(25, activation='softmax')(x)
model = keras.models.Model(base_model.input, x)

In [18]:
model.summary()

In [19]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'] # TODO
)

In [20]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
  train_dataset,
  epochs=100,
  validation_data=val_dataset,
  callbacks=[early_stopping_cb] # TODO
)

In [21]:
base_model = keras.applications.ResNet50(
    input_shape= (256, 256, 3),
    classes= 25,
    weights= 'imagenet',
    include_top= False,
    pooling= 'avg' # TODO
)

# Complete the model based on your choice of the architecture

base_model.trainable = False

for layer in base_model.layers[-5 : ]:
  layer.trainable = True

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(25, activation='softmax')(x)
model = keras.models.Model(base_model.input, x)

In [22]:
model.summary()

In [23]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'] # TODO
)

In [24]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
  train_dataset,
  epochs=100,
  validation_data=val_dataset,
  callbacks=[early_stopping_cb] # TODO
)

In [25]:
display_curves(history, 'loss')

In [26]:
display_curves(history, 'accuracy')

In [27]:
# TODO: evaluate the model on the validation dataset

loss, accuracy = model.evaluate(val_dataset)
print(accuracy)

In [28]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
  train_dataset,
  epochs=15,
  validation_data=val_dataset,
  callbacks=[early_stopping_cb] # TODO
)

In [29]:
base_model = keras.applications.ResNet50(
    input_shape= (256, 256, 3),
    classes= 25,
    weights= 'imagenet',
    include_top= False,
    pooling= 'avg' # TODO
)

# Complete the model based on your choice of the architecture

base_model.trainable = False

for layer in base_model.layers[-5 : ]:
  layer.trainable = True

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(25, activation='softmax')(x)
model = keras.models.Model(base_model.input, x)

In [30]:
model.summary()

In [31]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'] # TODO
)

In [32]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
  train_dataset,
  epochs=15,
  validation_data=val_dataset,
  callbacks=[early_stopping_cb] # TODO
)

In [33]:
display_curves(history, 'loss')

In [34]:
display_curves(history, 'accuracy')

In [35]:
# TODO: evaluate the model on the validation dataset

loss, accuracy = model.evaluate(val_dataset)
print(accuracy)

In [36]:
import os

test_ds = image_dataset_from_directory(
  directory= './test_data/',
  labels=None,
  color_mode='rgb',
  batch_size=64,
  image_size=(256, 256),
  interpolation='bilinear',
  shuffle = False
)

image_names = [img for img in test_dataset.file_paths]
print(image_names)

test_ds = test_ds.map(lambda x: (preprocess_input(x)))

predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = [int(class_names[pred]) for pred in predicted_classes]
predicted_artists = le.inverse_transform(predicted_classes)

submission = pd.DataFrame(columns=['file', 'artist'])
submission['file'] = file_names # TODO: get the file names from the test dataset
submission['artist'] = predicted_artists # TODO: get the predictions from the model (name of the painter)
submission

In [37]:
import os

test_ds = image_dataset_from_directory(
  directory= './test_data/',
  labels=None,
  color_mode='rgb',
  batch_size=64,
  image_size=(256, 256),
  interpolation='bilinear',
  shuffle = False
)

image_names = [img for img in test_ds.file_paths]
print(image_names)

test_ds = test_ds.map(lambda x: (preprocess_input(x)))

predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = [int(class_names[pred]) for pred in predicted_classes]
predicted_artists = le.inverse_transform(predicted_classes)

submission = pd.DataFrame(columns=['file', 'artist'])
submission['file'] = file_names # TODO: get the file names from the test dataset
submission['artist'] = predicted_artists # TODO: get the predictions from the model (name of the painter)
submission

In [38]:
import os

test_ds = image_dataset_from_directory(
  directory= './test_data/',
  labels=None,
  color_mode='rgb',
  batch_size=64,
  image_size=(256, 256),
  interpolation='bilinear',
  shuffle = False
)

image_names = [img.split('/')[-1].split('.')[0] for img in test_ds.file_paths]
test_ds = test_ds.map(lambda x: (preprocess_input(x)))

predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = [int(class_names[pred]) for pred in predicted_classes]
predicted_artists = le.inverse_transform(predicted_classes)

submission = pd.DataFrame(columns=['file', 'artist'])
submission['file'] = image_names # TODO: get the file names from the test dataset
submission['artist'] = predicted_artists # TODO: get the predictions from the model (name of the painter)
submission